In [1]:
import requests
import pandas as pd
import numpy
import time
import sys
import os
from dotenv import load_dotenv

from src.api.summoner_v4 import Summoner_v4
from src.api.match_v4 import Match_v4

from data_dragon import CHAMPION_NAME_TO_ID
from data_dragon import QUEUE_ID_TO_NAME

import datetime

In [2]:
load_dotenv()
API_KEY = os.getenv('API_KEY')
mv4 = Match_v4()
sv4 = Summoner_v4()

In [3]:
year = datetime.date.today().year
d = datetime.datetime(year, 1, 1)
d.timestamp()

1609488000.0

In [4]:
# Test to get season 11 RANKED statistics

# loop until not season 11 (started new year)
year = datetime.date.today().year
date = datetime.datetime(year, 1, 1)
NEW_YEAR_TIME_STAMP = date.timestamp()

# parameters Username QueueType (default ALL: None), championID(default All: None) 
username = "Halloween Jack"
queueType = 420 #ranked
championId = CHAMPION_NAME_TO_ID['malzahar']

total = 0
beginIndex= 0
endIndex=100
accountId = sv4.username_to_encryptedAccountID("Halloween Jack")
objects = []
while True:
    d = mv4.get_match_list(accountId,beginIndex=beginIndex,endIndex=endIndex)
    
    #check if d == -1
    
    
    matchList = d['matches']
    breakLoop = False
    for i in matchList:
        if i['timestamp']/1000 <= NEW_YEAR_TIME_STAMP: #conversion from ms to s
            breakLoop = True
            break
        elif ((championId is None or championId == i['champion']) and (queueType is None or queueType == i['queue'])):
            total += 1      
            # MAKE REQUEST GETMATCH//DATABASE HERE
            match = mv4.get_match(i['gameId'])
            
            if match == -1:
                breakLoop = True
                break
            
            #exclude remake case
            if match['gameDuration'] < 240:
                continue
            
            #find participantId for username
            numPlayers = 10
            participantInfo = match['participantIdentities']
            participantId = 0
            
            for j in participantInfo:
                if j['player']['accountId'] == accountId:
                    participantId = j['participantId']
            
            # riot uses 1 based indexing for array of participantstats
            participantStats = match['participants'][participantId-1]['stats']
            
            d = {
                "gameId":i['gameId'],
                "queueId":i['queue'],
                "champion":i['champion'],
                "lane":i['lane'],
                "timestamp":i['timestamp'],
                "CS":participantStats['totalMinionsKilled']+participantStats['neutralMinionsKilled'],
                "CS/min":(participantStats['totalMinionsKilled']+participantStats['neutralMinionsKilled'] )/(match['gameDuration']/60),
                "kills":participantStats['kills'],
                "deaths":participantStats['deaths'],
                "assists":participantStats['assists'],
                "goldEarned":participantStats['goldEarned'],
                "totalDamageDealtToChampions":participantStats['totalDamageDealtToChampions'],
                "damageDealtToObjectives":participantStats['damageDealtToObjectives'],
                "totalDamageTaken":participantStats['totalDamageTaken'],
                "firstBlood":participantStats['firstBloodKill'],
                "firstBloodAssist":participantStats['firstBloodAssist'],
                "win":participantStats['win']
            }
            objects.append(d)
            
    if breakLoop:
        break
    
    beginIndex += 100
    endIndex += 100

RATE LIMIT 429 error
sleeping
awake now
RATE LIMIT 429 error
sleeping
awake now


In [7]:
pd.DataFrame(objects)

,CS,CS/min,assists,champion,damageDealtToObjectives,deaths,firstBlood,firstBloodAssist,gameId,goldEarned,kills,lane,queueId,timestamp,totalDamageDealtToChampions,totalDamageTaken,win
0,131,6.055470,6,90,1191,1,False,False,3906102630,7369,3,MID,420,1620968543657,9964,3442,True
1,160,5.084746,16,90,9656,12,False,False,3878554223,12237,7,MID,420,1619224638386,35080,27754,False
2,177,5.633952,4,90,4433,8,False,False,3826300689,11387,5,MID,420,1615785505677,19828,18423,False
3,219,7.267699,6,90,6458,4,False,False,3825065497,10820,3,MID,420,1615706942972,19003,8914,True
4,195,7.447486,13,90,5851,2,False,False,3823440213,11345,2,TOP,420,1615620200244,18640,7472,True
5,149,5.793908,11,90,7325,4,False,False,3820305056,10318,5,MID,420,1615442099233,14728,10189,True
6,121,6.231760,1,90,2535,4,False,False,3817648943,5633,1,NONE,420,1615269631448,7263,8473,False
7,220,6.807633,10,90,7625,8,False,False,3817670138,14177,6,MID,420,1615267078174,27689,16519,True
8,159,6.538725,6,90,10770,3,False,False,3816365073,10171,5,MID,420,1615182002582,12242,9362,True
9,234,7.401160,7,90,407,4,False,False,3815817385,12716,5,MID,420,1615158887692,29873,18831,False
